# Visualize results

> An interactive notebook with IPywidgets to visualize model predictions and groundtruth.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [30]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from IPython.display import Image

In [3]:
from steel_segmentation.all import *

In [4]:
import os
import pandas as pd
import numpy as np

In [5]:
def submission_preproccessing(df: pd.DataFrame) -> pd.DataFrame:
    if not ("ImageId_ClassId" in df.columns):
        raise ValueError("Missing ImageId_ClassId column")
        
    splitted_cols = df["ImageId_ClassId"].str.split("_", expand=True)
    df["ImageId"], df["ClassId"] = splitted_cols[0], splitted_cols[1].astype("int64")
    return df

In [6]:
def read_submission(file_name:str = "ensemble_submission.csv") -> pd.DataFrame:
    df = pd.read_csv(sub_path / file_name)
    return submission_preproccessing(df)

In [7]:
file = "ensemble_submission.csv"
file_df = read_submission(file)
file_df.head()

,ImageId_ClassId,EncodedPixels,ImageId,ClassId
0,0000f269f.jpg_1,NaN,0000f269f.jpg,1
1,0000f269f.jpg_2,NaN,0000f269f.jpg,2
2,0000f269f.jpg_3,146418 5 146424 1 146577 30 146608 1 146610 1 146612 21 146638 48 146732 69 146802 1 146808 1 146810 1 146812 1 146820 125 146962 1 146976 225 147208 249 147461 252 147716 253 147971 254 148227 254 148483 254 148739 254 148995 254 149251 254 149507 254 149763 254 150019 254 150275 254 150532 253 150788 253 151045 252 151303 250 151563 246 151848 11 151862 13 151876 189 152152 169 152430 29 152489 1 152491 1 152493 6 152500 7 152557 12,0000f269f.jpg,3
3,0000f269f.jpg_4,NaN,0000f269f.jpg,4
4,000ccc2ac.jpg_1,NaN,000ccc2ac.jpg,1


Per visualizzare delle immagini di training selezionandole dall'elenco a discesa:

In [10]:
@interact_manual 
def show_images(file=os.listdir(str(train_path))):
    display(Image(train_path/file))

interactive(children=(Dropdown(description='file', options=('054190eb2.jpg', '73aa2aa79.jpg', '69f87ba66.jpg',…

Per visualizzare un campione di 5 immagini, selezionare dall'elenco a discesa la *submission* e successivamente la *ClassId* e infine il nome dell'immagine.

In [ ]:
subs =[file for file in os.listdir(str(sub_path)) if file.endswith(".csv")]

def read_submission_file(submission = subs) -> pd.DataFrame:
    df = pd.read_csv(sub_path / submission)
    return submission_preproccessing(df)

def select_classid(ClassId=[1,2,3,4]):
    df = submission.result
    cond_NaN = df.EncodedPixels.notna()
    cond_ClassId = df['ClassId'] == ClassId
    return df.loc[cond_NaN & cond_ClassId].sample(n=5)

In [80]:
submission = interactive(read_submission_file)
display(submission)

interactive(children=(Dropdown(description='submission', options=('resnet34_fastai_sub.csv', 'ensemble_submiss…

In [81]:
class_selection = interactive(select_classid)
display(class_selection)

interactive(children=(Dropdown(description='ClassId', options=(1, 2, 3, 4), value=1), Output()), _dom_classes=…

In [82]:
df = class_selection.result
submission = widgets.Dropdown(options=subs)
images = widgets.Dropdown(options=df.ImageId.to_list())

def update_imgids(*args):
    images.value = df.loc[cond].ImageId.to_list()

# Tie the image options to submission value
submission.observe(update_imgids, 'value')

# Show the images
def show_images(file):
    display(Image(test_path/file))

_ = interact(show_images, file=images)

interactive(children=(Dropdown(description='file', options=('2a5cc5f58.jpg', 'c4405bef9.jpg', '2b75eb7ae.jpg',…

Per visualizzare invece immagini dal training o test set:

In [83]:
# Create widgets
directory = widgets.Dropdown(options=[str(train_path), str(test_path)])
images = widgets.Dropdown(options=os.listdir(directory.value))

# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)
# Tie the image options to directory value
directory.observe(update_images, 'value')

# Show the images
def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

_ = interact(show_images, fdir=directory, file=images)

interactive(children=(Dropdown(description='fdir', options=('../data/train_images', '../data/test_images'), va…